# 텐서플로우에서 파이토치로 시계열 데이터 예측하기

## 목표

- 시계열 데이터를 다루는 텐서플로우 튜토리얼을 파이토치로 옮겨보기
- 파이토치로 시계열 데이터를 다루는 방법을 익히기

## 데이터셋
- 날씨 데이터셋 

In [1]:
from datasets import load_dataset
dataset = load_dataset("asanobm/jena_climate_2009_2016")
dataset

DatasetDict({
    train: Dataset({
        features: ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wx', 'Wy', 'max Wx', 'max Wy', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
        num_rows: 49063
    })
    validation: Dataset({
        features: ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wx', 'Wy', 'max Wx', 'max Wy', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
        num_rows: 14018
    })
    test: Dataset({
        features: ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'Wx', 'Wy', 'max Wx', 'max Wy', 'Day sin', 'Day cos', 'Year sin', 'Year cos'],
        num_rows: 7010
    })
})

In [2]:
train_df = dataset['train'].to_pandas()
val_df = dataset['validation'].to_pandas()
test_df = dataset['test'].to_pandas()

## 데이터 창 작업

연속된 샘플 윈도우를 만들어 일련의 예측을 수행한다.

입력 윈도우의 주요 특성

* 입력 및 레이블 윈도우의 너비 (타임스텝 수)
* 입력, 레이블 또는 둘 모두로 사용되는 특성

## 모델

* 단일 출력 및 다중 출력 예측
* 단일 타임스텝 및 다중 타임스텝 예측



In [3]:
# Window Generator
import matplotlib.pyplot as plt
import numpy as np
import torch

class WindowGenerator():
    """
    시계열 데이터를 위한 윈도우 생성기 클래스입니다.
    이 클래스는 주어진 입력 폭, 라벨 폭, 이동 크기 등을 기반으로
    학습, 검증, 테스트 데이터프레임에서 윈도우를 생성합니다.
    Attributes:
        train_df (pd.DataFrame): 학습 데이터프레임.
        val_df (pd.DataFrame): 검증 데이터프레임.
        test_df (pd.DataFrame): 테스트 데이터프레임.
        label_columns (list of str): 라벨 열 이름 리스트.
        label_columns_indices (dict): 라벨 열 이름과 인덱스의 딕셔너리.
        column_indices (dict): 모든 열 이름과 인덱스의 딕셔너리.
        input_width (int): 입력 폭.
        label_width (int): 라벨 폭.
        shift (int): 이동 크기.
        total_window_size (int): 전체 윈도우 크기.
        input_slice (slice): 입력 슬라이스 객체.
        input_indices (np.ndarray): 입력 인덱스 배열.
        label_start (int): 라벨 시작 인덱스.
        labels_slice (slice): 라벨 슬라이스 객체.
        label_indices (np.ndarray): 라벨 인덱스 배열.
    Methods:
        __repr__():
        split_window(features):
            주어진 특징 배열을 입력과 라벨로 분할합니다.
            Args:
                features (np.ndarray): 특징 배열.
                tuple: 입력과 라벨 배열의 튜플.

    """
    def __init__(self,
                 input_width,
                 label_width,
                 shift,
                 train_df,
                 val_df,
                 test_df,
                 label_columns=None,
                 ):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                           enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        """
        객체의 문자열 표현을 반환합니다.

        반환되는 문자열은 다음 정보를 포함합니다:
        - 전체 윈도우 크기
        - 입력 인덱스
        - 라벨 인덱스
        - 라벨 열 이름(들)

        Returns:
            str: 객체의 문자열 표현
        """
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = torch.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information in TensorFlow,
        # but this is not needed for PyTorch tensors.

        return inputs, labels
    
    def make_dataset(self, data):
        """
        주어진 데이터를 PyTorch 데이터셋으로 변환합니다.

        Args:
            data (array-like): 변환할 데이터. float32 타입의 numpy 배열로 변환됩니다.

        Returns:
            DataLoader: 변환된 데이터셋을 포함하는 DataLoader 객체. 배치 크기는 32이며, 셔플이 적용됩니다.
        """
        data = np.array(data, dtype=np.float32)
        ds = torch.utils.data.TensorDataset(torch.tensor(data, dtype=torch.float32))
        ds = torch.utils.data.DataLoader(ds, batch_size=32, shuffle=True)
        ds = ds.map(self.split_window)
        return ds
        
    
    def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
        """
        모델의 예측 결과를 시각화하는 함수입니다.
        Args:
            model (tf.keras.Model, optional): 예측을 수행할 모델입니다. 기본값은 None입니다.
            plot_col (str, optional): 시각화할 열의 이름입니다. 기본값은 'T (degC)'입니다.
            max_subplots (int, optional): 최대 서브플롯의 개수입니다. 기본값은 3입니다.
        Example:
            window.plot(model=my_model, plot_col='T (degC)', max_subplots=3)
        Notes:
            - 입력 데이터와 레이블 데이터를 시각화합니다.
            - 모델이 제공된 경우, 모델의 예측 결과도 시각화합니다.
            - 최대 서브플롯의 개수는 max_subplots 인수로 조정할 수 있습니다.
        """
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue
            
            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                            marker='X', edgecolors='k', label='Predictions',
                            c='#ff7f0e', s=64)
                
            if n == 0:
                plt.legend()


    @property
    def train(self):
        return self.make_dataset(self.train_df)
    
    @property
    def val(self):
        return self.make_dataset(self.val_df)
    
    @property
    def test(self):
        return self.make_dataset(self.test_df)
    
    @property
    def example(self):
        """
        학습 데이터셋에서 예제 배치를 반환합니다.

        Returns:
            tuple: 입력 데이터와 레이블 데이터의 튜플.
        """
        result = next(iter(self.train))
        return result

In [4]:
w1 = WindowGenerator(
    train_df=train_df, 
    val_df=val_df,
    test_df=test_df,
    input_width=24, 
    label_width=1,
    shift=24,
    label_columns=['T (degC)']
    )
w1

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [47]
Label column name(s): ['T (degC)']

In [5]:
w2 = WindowGenerator(
    train_df=train_df, 
    val_df=val_df,
    test_df=test_df,
    input_width=6, 
    label_width=1,
    shift=1,
    label_columns=['T (degC)']
    )
w2

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [6]
Label column name(s): ['T (degC)']

In [6]:
# Stack three slices, the length of the total window.
example_window = torch.stack([
        torch.tensor(train_df[:w2.total_window_size].values),
        torch.tensor(train_df[100:100+w2.total_window_size].values),
        torch.tensor(train_df[200:200+w2.total_window_size].values),
        ]
)

example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

All shapes are: (batch, time, features)
Window shape: torch.Size([3, 7, 19])
Inputs shape: torch.Size([3, 6, 19])
Labels shape: torch.Size([3, 1, 1])


### 시각화

In [10]:
w2.plot()

AttributeError: 'DataLoader' object has no attribute 'map'